In [1]:
import os

In [2]:
print('Process (%s) start...' % os.getpid())

pid = os.fork() 
if pid == 0:
    print('I am child process (%s) and my parent is %s' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (2719) start...
I (2719) just created a child process (2787).
I am child process (2787) and my parent is 2719


In [3]:
# 有了fork调用，一个进程接到新任务时，就可以复制一个子进程来处理新任务

In [4]:
# 在Windows下使用multiprocessing模块中的Process类来代表一个进程对象

In [5]:
from multiprocessing import Process
import os 

In [7]:
# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))
    
print('Parent process %s. ' % os.getpid())
p = Process(target=run_proc, args=('test',))
print('Child process will start.')
p.start()
p.join()
print('Child process end.')

Parent process 2719. 
Child process will start.
Run child process test (3994)...
Child process end.


In [8]:
# 启动大量子线程，可以用线程池的方式创建

In [10]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' %(name, (end-start)))
    
print('Parent process %s.' % os.getpid())
p = Pool(5)
for i in range(5):
    p.apply_async(long_time_task, args=(i,))
print('Waiting for all subprocesse done...')
p.close()
p.join()
print('All subprocesses done.')

Parent process 2719.
Run task 1 (4746)...
Run task 0 (4747)...
Run task 4 (4750)...
Run task 2 (4749)...
Run task 3 (4748)...
Waiting for all subprocesse done...
Task 2 runs 0.26 seconds.
Task 1 runs 0.71 seconds.
Task 4 runs 0.78 seconds.
Task 0 runs 1.21 seconds.
Task 3 runs 1.51 seconds.
All subprocesses done.


In [11]:
# Pool对象调用join()方法会等待所有子进程执完毕，而且join()调用前必须先调用close()，调用close()后不能继续添加新的进程
# p = Pool() 默认大小为CPU的核数，如果不传入5，则会看到'Run task ? (????)' 不同时执行，会出现等待的效果，可以将5去掉重新执行看效果

In [12]:
# 子进程 使用subprocess模块可以启动一个子线程，然后控制其输入输出

In [15]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [16]:
# 进程间通信
# Process之间有多种通信方式，Python的multiProcessing模块封装了底层的机制，提供Queue、Pipes等多种方式狡猾数据

In [17]:
from multiprocessing import Process, Queue
import os, time, random

In [23]:
# 写数据进程
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())
        
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' %value)

In [24]:
# 父进程创建Queue，并传给各个子进程
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# 启动子进程pw,写入：
pw.start()
# 启动子进程pr,读取：
pr.start()
# 等待pw结束
pw.join()
# pr是死循环，只能强制终止
pr.terminate()

Process to write: 6756
Put A to queue...
Process to read: 6759
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


In [25]:
# 进程小结
# 1.在Unix/Linux/Mac下，使用fork()调用实现多进程
# 2.要实现跨平台的多进程，可以使用multiprocessing模块
# 3.进程间的通信通过 Queue, Pipes等实现

In [26]:
# Python支持多线程变成，提供了两个模块：_thread 和 threading，后者对前者进行了封装，使用 threading 即可。
# 启动线程的方式：1.把函数传入Thread实例，然后调用start()

In [27]:
import time, threading

In [29]:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target = loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


In [30]:
# threading模块有个current_thread()函数，他返回当前线程的实例。
# 主线程的名字叫 MainThread，子线程的名字在创建时指定，如果没指定，系统默认给Thread-1，Thread-2

In [31]:
# 多线程的危险之处：共享数据
# 所有变量由所有线程共享，任何一个变量都可以被任何一个线程修改，因此，线程之间共享数据会出现同时修改数据的可能性。
# 所以要引入线程锁 Lock

In [37]:
# 以银行存款为例
import time, threading

# 假定你的银行存款为balance
balance = 0

def change_it(n) :
    global balance
    balance = balance + n
    balance = balance - n
    # 理论上正常执行下来 balance必然为0
    
def run_thread(n):
    for i in range(1000000):
        change_it(n)
        
t1 = threading.Thread(target=run_thread, args=(5,)) # balance + 5 然后 - 5
t2 = threading.Thread(target=run_thread, args=(8,)) # balacne + 8 然后 - 8
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

16


In [38]:
# 由于线程的调用是由操作系统决定的，一条语句在转换为机器语言是会变成若干条语句
# 因此无法保证，线程t1在对balance做更改时，t2线程不对balance做更改
# 所以要确保一个线程在修改balance时，其他线程一定不能修改balance

In [39]:
# 创建一把锁，然所有线程公用，只有一个线程能拥有锁，其他线程只有等待到锁释放时才能使用

In [40]:
# 以银行存款为例
import time, threading

# 假定你的银行存款为balance
balance = 0
lock = threading.Lock()

def change_it(n) :
    global balance
    balance = balance + n
    balance = balance - n
    # 理论上正常执行下来 balance必然为0
    
def run_thread(n):
    for i in range(1000000):
        # 获取锁
        lock.acquire()
        try:
            change_it(n)
        finally:
            lock.release()
        
t1 = threading.Thread(target=run_thread, args=(5,)) # balance + 5 然后 - 5
t2 = threading.Thread(target=run_thread, args=(8,)) # balacne + 8 然后 - 8
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


In [41]:
# 当多个线程同时执行 lock.acquire()时，只有一个线程能成功获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止
# 获得锁的线程用完后一定要释放，否则其他等待锁的线程将永远等待下去，成为死线程。因此使用try...finally确保锁一定会被释放

In [42]:
# Python一个进程最多占用一个核，与C/Java不同，他们可以建多个线程占用多个CPU
# 这是Python的历史遗留问题，一个进程中会有一个GIL（Global Interpreter Lock）用于分配系统资源。
# Python多线程充分利用资源的问题，需要通过多进程来解决